In [8]:
import pandas as pd
import json
import os

In [9]:
dir_name = 'C:/Users/KIHyuk/Desktop/brunch_data/json'

def get_file_list(dir_name): # file name들을 가져오는 함수 # 폴더명 인자 # 폴더가 위치한 경로를 인자로
    return os.listdir(dir_name) # 폴더 내 파일명을 리스트 형태로 반환 

file_list = get_file_list(dir_name)

In [10]:
def pre_keyword(x):
    tmp = []
    for val in x:
        tmp.append(val.replace("\n","").replace(" ",""))
    return tmp

def pre_comment(x):
    if len(x) == 0:
        return None
    else :
        return x

def pre_text(x):
    return str(x)

class_condition = {'지구한바퀴_세계여행':0 , '그림·웹툰':1, '시사·이슈':2, 'IT_트렌드':3, '사진·촬영':4, '취향저격_영화_리뷰':5,
                   '뮤직_인사이드':6, '육아_이야기':7, '요리·레시피':8, '건강·운동':9, '멘탈_관리_심리_탐구':10, '문화·예술':11, '건축·설계':12,
                   '인문학·철학':13, '쉽게_읽는_역사':14, '우리집_반려동물':15, '글쓰기_코치':16, '오늘은_이런_책':16, '직장인_현실_조언':17, '스타트업_경험담':17,
                   '디자인_스토리':18, '멋진_캘리그래피':18, '사랑·이별':19, '감성_에세이':19}

In [14]:
all_df = pd.DataFrame(columns=['class','text'])
each_df = {}

for file in file_list:
    with open('C:/Users/KIHyuk/Desktop/brunch_data/json/'+file,encoding='UTF8') as json_file:
        json_data = json.load(json_file)
    
    df = pd.DataFrame(json_data['data'],
                  columns=['title','keyword','text','nickname','publish_date','likes','share','comment','url','url_plink'])
    df = df.dropna(subset=['text'])
    df['keyword'] = df['keyword'].apply(pre_keyword)
    df['comment'] = df['comment'].apply(pre_comment)
    df['text'] = df['text'].apply(pre_text)
    df.insert(0,"class",file[:-5])
#     df['class'] = df['class'].map(class_condition)
    
    all_df = pd.concat([all_df,df[['class','text','keyword','publish_date','likes','share','comment','url']][:100]])
    each_df[file[:-5]] = df
    
all_df = all_df.reset_index(drop=True)

def split_datetime(x):
    return x.split('T')[0]

all_df['publish_date'] = all_df['publish_date'].apply(split_datetime)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [13]:
all_df

,class,keyword,publish_date,text
0,IT_트렌드,"[서핑, 넷플릭스, 모바일]",2020-03-30,['요즘 집콕하는 시간이 길어지다 보니 넷플릭스에 접속하는 시간이 길어졌다. 그렇다...
1,IT_트렌드,"[코로나, 재택근무, 애플]",2020-03-30,"['', ""타겟 유저, 타겟 시장이라는 용어를 흔히들 씁니다. 신사업을 기획하거나 ..."
2,IT_트렌드,"[디자인, UX, UI]",2020-03-30,"['', '-지난글 보기-', 'https://brunch.co.kr/@sei0/8..."
3,IT_트렌드,"[애플, 안경, 아이패드프로]",2020-03-30,['텐서플로우 개발자 서밋에\xa0이어 두 번째로 인공지능 기조연설\xa0시리즈를 ...
4,IT_트렌드,"[지니, UI디자인, UX]",2020-03-30,['작업 초반에는 태블릿에 앱을 탑재해 적용된 화면을 보며 진행하였기에 큰 문제없이...
...,...,...,...,...
2395,취향저격_영화_리뷰,"[예술, 영화, 존재]",2020-03-27,"[""예술경영인의 눈으로 바라보는 '존재'의 의미 예술영화<벌새>를 통하여"", '',..."
2396,취향저격_영화_리뷰,"[영화, 인정, 주인공]",2020-03-27,"['', '뉴스를 틀면 늘 한숨부터 나옵니다. 세상엔 어쩜 이렇게 나쁜 사람도, 속..."
2397,취향저격_영화_리뷰,"[영화, 영화리뷰, 공감에세이]",2020-03-27,"['', '\xa0잘 만든 영화라는 느낌을 받을 때가 있다. 색감이 완벽하거나, 배..."
2398,취향저격_영화_리뷰,"[모리, 영화, 리뷰]",2020-03-27,['두 지팡이를 힘겹게 짚고 걷던 구마가이 모리카즈(야마자키 츠토무)는 길가에 난 ...


In [115]:
def keyword_trend_chart(df,select_keyword,resampling_num):    
    if resampling_num == 'M':
        df = df['keyword']['2020-01-01':].resample('M').sum()
    else:
        df = df['keyword']['2020-01-01':].resample(resampling_num + 'D').sum()
    
    res_df = pd.DataFrame(columns=select_keyword,index=df.index)
    for keyword in select_keyword:
        keyword_week_count = []
        for week in range(len(df)):
            keyword_week_count.append(df.iloc[week].count(keyword))
        res_df[keyword] = keyword_week_count
        
    return res_df

In [118]:
keyword_trend_chart(all_df_copy,["감성에세이","여행"],'14')

,감성에세이,여행
publish_date,,
2020-01-01,1,0
2020-01-15,2,0
2020-01-29,3,0
2020-02-12,8,22
2020-02-26,39,39
2020-03-11,52,69
2020-03-25,116,44
2020-04-08,0,0


In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(all_df['text'],all_df['class'],test_size=0.2 ,shuffle=True)

In [28]:
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

import re

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

train_data = []
for sentence in X_train[:5]:
#     temp_X = []
    sentence = re.sub(r'[^0-9a-zA-Zㄱ-힗]',' ',sentence)
    sentence = re.sub(r'[xa0]','',sentence)
#     temp_X = okt.morphs(sentence, stem=True) # 토큰화
#     temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    train_data.append(sentence)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(X_train)
tfidf_matrix_train = tfidf_vect.transform(X_train)

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
lg_clf = LogisticRegression(random_state=0)

params = {'C' :[1, 3.5, 4.5, 5.5, 10]}

grid_cv = GridSearchCV(lg_clf, parma_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train,y_train)
print(grid_cv.best_params_, round(grid_cv.best_score_,4))